### Adjust Word Group In Youtube Sentence Word Usage Analysis

In [52]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [53]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [54]:
# language pair
lang_folder = "Dutch"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# native word select for Part 2 
word_start = 0  # 0  # native word start index
word_end = 200 # 200 # native word end index

# word all usage in sent
word_use_num_max = 7  # 5
word_use_num_min = 1  # 1

# youtube sentence usage for remain word 
youtube_sent_usage = "selected"  # "all","selected"

In [55]:
#path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
#Talk Time/Talk Time 1/Result/2-Adjust Word Group In Youtube Sentence Word Usage Analysis"
#
#Path(path).mkdir(parents=True, exist_ok=True)

In [56]:
def word_in_wordgroup_simple(source_word_list, df_target, target_column, word_sample_num):

    '''word_in_wordgroup_simple(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number. 
    '''
    
    df_select = df_target[[f"{target_column}"]].dropna()
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,"word",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

#### Word Usage

In [57]:
def word_index(df_word, word_column, sentence):
    word_index_list = []
    word_index_dict = {var2:var1 for (var1, var2) in enumerate(df_word[f"{word_column}"])}
    words = word_tokenize(sentence)
    for word in words:
        value = word_index_dict[f"{word}"]
        word_index_list.append(value)
    return word_index_list

In [58]:
disable_video_id_list = ["H6E6N70jYqI","BoaYsdPtJYA","HUZINsU40Fk","Et3diPcEmfY"]

In [59]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,ik,22539531
1,je,20769946
2,het,15696161
3,de,15258816
4,dat,13387137
...,...,...
331769,oorlogsgezichten,5
331770,opdrachtenlijst,5
331771,verlsaafde,5
331772,oxidatieproces,5


In [60]:
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select.reset_index(inplace=True)
df_word_select

,index,word,frequency
0,0,ik,22539531
1,1,je,20769946
2,2,het,15696161
3,3,de,15258816
4,4,dat,13387137
...,...,...,...
195,195,genoeg,353128
196,196,wist,352948
197,197,oh,351988
198,198,klaar,350506


In [61]:
df_youtube_sent_result = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Talk Time/Data/\
1-Word Group In Youtube Sentence/{lang_folder.lower().capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word.csv")
df_youtube_sent_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,de n,291,294,tegenwoordig de n,EHLs51vEPuU,https://www.youtube.com/watch?v=EHLs51vEPuU&t=...
1,ik s,20,22,dan zet ik s nachts om,EL02PyHRmOU,https://www.youtube.com/watch?v=EL02PyHRmOU&t=20s
2,u de,94,95,wanneer u de comfort seat niet gebruikt moet h...,7KsIU2XtYo8,https://www.youtube.com/watch?v=7KsIU2XtYo8&t=94s
3,of n,731,732,eenenvijftig of n,WlJ12LESGP0,https://www.youtube.com/watch?v=WlJ12LESGP0&t=...
4,of t,19,20,het rijksmonument sur meuse bijvoorbeeld of t ...,mMvNkD7AJEk,https://www.youtube.com/watch?v=mMvNkD7AJEk&t=19s
...,...,...,...,...,...,...
220708,dat er voor ja moet er geen gaan we nou ja maa...,694,700,jarenlang dat er voor ja moet er geen gaan we ...,9rsOGla4V6s,https://www.youtube.com/watch?v=9rsOGla4V6s&t=...
220709,en de huis gaan en ik hou gewoon heel erg veel...,207,213,leger en de huis gaan en ik hou gewoon heel er...,V1RX2oPECHY,https://www.youtube.com/watch?v=V1RX2oPECHY&t=...
220710,als ik het niet ben ik zeggen jaar dan heb jij...,2017,2024,lieg als ik het niet ben ik zeggen jaar dan he...,iLJg6UP0y_8,https://www.youtube.com/watch?v=iLJg6UP0y_8&t=...
220711,niet zeggen denk en ik het goed gedaan maar di...,742,746,niet zeggen denk en ik het goed gedaan maar di...,Pg4VBpojtGc,https://www.youtube.com/watch?v=Pg4VBpojtGc&t=...


In [62]:
df_youtube_sent_result["word_index_list"] = df_youtube_sent_result["search_string"].apply(lambda sent: word_index(df_word_select, "word", sent))
df_youtube_sent_result

,search_string,start_time,end_time,sentence,video_id,video_url,word_index_list
0,de n,291,294,tegenwoordig de n,EHLs51vEPuU,https://www.youtube.com/watch?v=EHLs51vEPuU&t=...,"[3, 26]"
1,ik s,20,22,dan zet ik s nachts om,EL02PyHRmOU,https://www.youtube.com/watch?v=EL02PyHRmOU&t=20s,"[0, 94]"
2,u de,94,95,wanneer u de comfort seat niet gebruikt moet h...,7KsIU2XtYo8,https://www.youtube.com/watch?v=7KsIU2XtYo8&t=94s,"[33, 3]"
3,of n,731,732,eenenvijftig of n,WlJ12LESGP0,https://www.youtube.com/watch?v=WlJ12LESGP0&t=...,"[65, 26]"
4,of t,19,20,het rijksmonument sur meuse bijvoorbeeld of t ...,mMvNkD7AJEk,https://www.youtube.com/watch?v=mMvNkD7AJEk&t=19s,"[65, 77]"
...,...,...,...,...,...,...,...
220708,dat er voor ja moet er geen gaan we nou ja maa...,694,700,jarenlang dat er voor ja moet er geen gaan we ...,9rsOGla4V6s,https://www.youtube.com/watch?v=9rsOGla4V6s&t=...,"[4, 19, 21, 48, 44, 19, 41, 63, 11, 106, 48, 1..."
220709,en de huis gaan en ik hou gewoon heel erg veel...,207,213,leger en de huis gaan en ik hou gewoon heel er...,V1RX2oPECHY,https://www.youtube.com/watch?v=V1RX2oPECHY&t=...,"[8, 3, 153, 63, 8, 0, 128, 121, 141, 174, 113,..."
220710,als ik het niet ben ik zeggen jaar dan heb jij...,2017,2024,lieg als ik het niet ben ik zeggen jaar dan he...,iLJg6UP0y_8,https://www.youtube.com/watch?v=iLJg6UP0y_8&t=...,"[25, 0, 2, 7, 28, 0, 126, 150, 35, 24, 38, 2, ..."
220711,niet zeggen denk en ik het goed gedaan maar di...,742,746,niet zeggen denk en ik het goed gedaan maar di...,Pg4VBpojtGc,https://www.youtube.com/watch?v=Pg4VBpojtGc&t=...,"[7, 126, 90, 8, 0, 2, 50, 142, 18, 30, 39, 0, ..."


In [63]:
df_youtube_sent_result = df_youtube_sent_result[~df_youtube_sent_result["video_id"].isin(disable_video_id_list)]
df_youtube_sent_result

,search_string,start_time,end_time,sentence,video_id,video_url,word_index_list
0,de n,291,294,tegenwoordig de n,EHLs51vEPuU,https://www.youtube.com/watch?v=EHLs51vEPuU&t=...,"[3, 26]"
1,ik s,20,22,dan zet ik s nachts om,EL02PyHRmOU,https://www.youtube.com/watch?v=EL02PyHRmOU&t=20s,"[0, 94]"
2,u de,94,95,wanneer u de comfort seat niet gebruikt moet h...,7KsIU2XtYo8,https://www.youtube.com/watch?v=7KsIU2XtYo8&t=94s,"[33, 3]"
3,of n,731,732,eenenvijftig of n,WlJ12LESGP0,https://www.youtube.com/watch?v=WlJ12LESGP0&t=...,"[65, 26]"
4,of t,19,20,het rijksmonument sur meuse bijvoorbeeld of t ...,mMvNkD7AJEk,https://www.youtube.com/watch?v=mMvNkD7AJEk&t=19s,"[65, 77]"
...,...,...,...,...,...,...,...
220708,dat er voor ja moet er geen gaan we nou ja maa...,694,700,jarenlang dat er voor ja moet er geen gaan we ...,9rsOGla4V6s,https://www.youtube.com/watch?v=9rsOGla4V6s&t=...,"[4, 19, 21, 48, 44, 19, 41, 63, 11, 106, 48, 1..."
220709,en de huis gaan en ik hou gewoon heel erg veel...,207,213,leger en de huis gaan en ik hou gewoon heel er...,V1RX2oPECHY,https://www.youtube.com/watch?v=V1RX2oPECHY&t=...,"[8, 3, 153, 63, 8, 0, 128, 121, 141, 174, 113,..."
220710,als ik het niet ben ik zeggen jaar dan heb jij...,2017,2024,lieg als ik het niet ben ik zeggen jaar dan he...,iLJg6UP0y_8,https://www.youtube.com/watch?v=iLJg6UP0y_8&t=...,"[25, 0, 2, 7, 28, 0, 126, 150, 35, 24, 38, 2, ..."
220711,niet zeggen denk en ik het goed gedaan maar di...,742,746,niet zeggen denk en ik het goed gedaan maar di...,Pg4VBpojtGc,https://www.youtube.com/watch?v=Pg4VBpojtGc&t=...,"[7, 126, 90, 8, 0, 2, 50, 142, 18, 30, 39, 0, ..."


In [64]:
#df_youtube_sent_select.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_List.xlsx", index=False)

In [65]:
df_youtube_sent_select = df_youtube_sent_result[df_youtube_sent_result["start_time"] <= 700]
df_youtube_sent_select.reset_index(drop=True, inplace=True)
df_youtube_sent_select

,search_string,start_time,end_time,sentence,video_id,video_url,word_index_list
0,de n,291,294,tegenwoordig de n,EHLs51vEPuU,https://www.youtube.com/watch?v=EHLs51vEPuU&t=...,"[3, 26]"
1,ik s,20,22,dan zet ik s nachts om,EL02PyHRmOU,https://www.youtube.com/watch?v=EL02PyHRmOU&t=20s,"[0, 94]"
2,u de,94,95,wanneer u de comfort seat niet gebruikt moet h...,7KsIU2XtYo8,https://www.youtube.com/watch?v=7KsIU2XtYo8&t=94s,"[33, 3]"
3,of t,19,20,het rijksmonument sur meuse bijvoorbeeld of t ...,mMvNkD7AJEk,https://www.youtube.com/watch?v=mMvNkD7AJEk&t=19s,"[65, 77]"
4,ja u,129,130,ja u ziet het goed generaal maatennaaier is terug,3fAvv1RgbUQ,https://www.youtube.com/watch?v=3fAvv1RgbUQ&t=...,"[48, 33]"
...,...,...,...,...,...,...,...
127523,we hebben echt heel veel mensen nodig om dit t...,392,397,we hebben echt heel veel mensen nodig om dit t...,g01V_H0TU8o,https://www.youtube.com/watch?v=g01V_H0TU8o&t=...,"[11, 49, 92, 141, 113, 115, 114, 29, 30, 15, 5..."
127524,dus ja ik denk zeker dat het heel anders is en...,265,269,dus ja ik denk zeker dat het heel anders is en...,eldUSfVfYMc,https://www.youtube.com/watch?v=eldUSfVfYMc&t=...,"[88, 48, 0, 90, 146, 4, 2, 141, 167, 5, 8, 0, ..."
127525,die we nu gaan doen maar ik wilde er wel even ...,234,240,die we nu gaan doen maar ik wilde er wel even ...,CqaNlQx_CmE,https://www.youtube.com/watch?v=CqaNlQx_CmE&t=...,"[20, 11, 54, 63, 58, 18, 0, 159, 19, 47, 111, ..."
127526,dat er voor ja moet er geen gaan we nou ja maa...,694,700,jarenlang dat er voor ja moet er geen gaan we ...,9rsOGla4V6s,https://www.youtube.com/watch?v=9rsOGla4V6s&t=...,"[4, 19, 21, 48, 44, 19, 41, 63, 11, 106, 48, 1..."


In [66]:
# # word usage in max condition
# result_list = []
# index_list = []
# for i in range(len(df_youtube_sent_select)):
#     search = df_youtube_sent_select.loc[i,"search_string"]
#     start=df_youtube_sent_select.loc[i,"start_time"]
#     end=df_youtube_sent_select.loc[i,"end_time"]
#     sent=df_youtube_sent_select.loc[i,"sentence"]
#     id = df_youtube_sent_select.loc[i,"video_id"]
#     id_url = df_youtube_sent_select.loc[i,"video_url"]
#     word_index = df_youtube_sent_select.loc[i,"word_index_list"]
#     list_var = index_list + word_index
#     #list_var = index_list + [x for x in word_index]
# 
#     dict_list_count = Counter(list_var)
#     count_list = list(dict_list_count.values())
# 
#     if any([True if i>word_use_num_max else False for i in count_list]):
#         index_list = index_list
#     else:
#         index_list = index_list + word_index
#         result_list.append([search,start,end,sent,id,id_url,word_index])
#     

In [67]:
# word usage in min max condition
word_num_index_dict = {}
for i in df_word_select["index"]:
    word_num_index_dict[i] = 0

result_list = []
index_list = []
for i in range(len(df_youtube_sent_select)):
    search = df_youtube_sent_select.loc[i,"search_string"]
    start=df_youtube_sent_select.loc[i,"start_time"]
    end=df_youtube_sent_select.loc[i,"end_time"]
    sent=df_youtube_sent_select.loc[i,"sentence"]
    id = df_youtube_sent_select.loc[i,"video_id"]
    id_url = df_youtube_sent_select.loc[i,"video_url"]
    word_index = df_youtube_sent_select.loc[i,"word_index_list"]
    list_var = index_list + word_index
    #list_var = index_list + [x for x in word_index]
    # word count for max
    dict_list_count = Counter(list_var)
    count_list = list(dict_list_count.values())
     # word count for min
    count_list2 = list(word_num_index_dict.values())   

    if any([True if i>word_use_num_max else False for i in count_list]) or not(any([True if j<word_use_num_min else False for j in count_list2])):
        index_list = index_list
    else:
        index_list = index_list + word_index
        result_list.append([search,start,end,sent,id,id_url,word_index])

        for item2 in dict_list_count.items(): 
            word_num_index_dict[item2[0]] = item2[1]

print(f"Max condition: {any([True if i>word_use_num_max else False for i in count_list2])} \nMin condition: {any([True if j<word_use_num_min else False for j in count_list2])}") 

Max condition: False 
Min condition: True


In [68]:
#word_num_index_dict

In [69]:
set_index = set(index_list)
set_all_index = set([x for x in df_word_select["index"]])

In [70]:
not_in_sent = set_all_index.difference(set_index)
not_in_sent = list(not_in_sent)
not_in_sent

[160, 193, 194, 163, 196, 199, 169, 139, 176, 189, 151, 154, 155, 157, 62]

In [71]:
#Counter(index_list)

In [72]:
df_youtube_sent_sample = pd.DataFrame(result_list, columns=["search_string","start_time","end_time","sentence","video_id","video_url","word_index"])
df_youtube_sent_sample

,search_string,start_time,end_time,sentence,video_id,video_url,word_index
0,de n,291,294,tegenwoordig de n,EHLs51vEPuU,https://www.youtube.com/watch?v=EHLs51vEPuU&t=...,"[3, 26]"
1,ik s,20,22,dan zet ik s nachts om,EL02PyHRmOU,https://www.youtube.com/watch?v=EL02PyHRmOU&t=20s,"[0, 94]"
2,u de,94,95,wanneer u de comfort seat niet gebruikt moet h...,7KsIU2XtYo8,https://www.youtube.com/watch?v=7KsIU2XtYo8&t=94s,"[33, 3]"
3,of t,19,20,het rijksmonument sur meuse bijvoorbeeld of t ...,mMvNkD7AJEk,https://www.youtube.com/watch?v=mMvNkD7AJEk&t=19s,"[65, 77]"
4,ja u,129,130,ja u ziet het goed generaal maatennaaier is terug,3fAvv1RgbUQ,https://www.youtube.com/watch?v=3fAvv1RgbUQ&t=...,"[48, 33]"
...,...,...,...,...,...,...,...
520,moeten misschien,178,179,dus klanten moeten misschien eventjes hun fiet...,qVoH2QwwDWg,https://www.youtube.com/watch?v=qVoH2QwwDWg&t=...,"[83, 103]"
521,niets anders kon,102,103,omdat ie op dat moment niets anders kon bedenken,dw4uOU5fUEY,https://www.youtube.com/watch?v=dw4uOU5fUEY&t=...,"[109, 167, 186]"
522,komt alleen omdat,431,433,is gekomen komt alleen omdat verschillende ond...,xKNDkrAQ3iM,https://www.youtube.com/watch?v=xKNDkrAQ3iM&t=...,"[110, 93, 138]"
523,zeg gewoon bedankt,549,551,zeg gewoon bedankt,UltPsmBVTV0,https://www.youtube.com/watch?v=UltPsmBVTV0&t=...,"[130, 121, 170]"


In [73]:
df_youtube_sent_sample["video_url"] = "https://www.youtube.com/watch?v="+df_youtube_sent_sample['video_id'].map(str)+"&t="+df_youtube_sent_sample['start_time'].map(str)+"s"
df_youtube_sent_sample

,search_string,start_time,end_time,sentence,video_id,video_url,word_index
0,de n,291,294,tegenwoordig de n,EHLs51vEPuU,https://www.youtube.com/watch?v=EHLs51vEPuU&t=...,"[3, 26]"
1,ik s,20,22,dan zet ik s nachts om,EL02PyHRmOU,https://www.youtube.com/watch?v=EL02PyHRmOU&t=20s,"[0, 94]"
2,u de,94,95,wanneer u de comfort seat niet gebruikt moet h...,7KsIU2XtYo8,https://www.youtube.com/watch?v=7KsIU2XtYo8&t=94s,"[33, 3]"
3,of t,19,20,het rijksmonument sur meuse bijvoorbeeld of t ...,mMvNkD7AJEk,https://www.youtube.com/watch?v=mMvNkD7AJEk&t=19s,"[65, 77]"
4,ja u,129,130,ja u ziet het goed generaal maatennaaier is terug,3fAvv1RgbUQ,https://www.youtube.com/watch?v=3fAvv1RgbUQ&t=...,"[48, 33]"
...,...,...,...,...,...,...,...
520,moeten misschien,178,179,dus klanten moeten misschien eventjes hun fiet...,qVoH2QwwDWg,https://www.youtube.com/watch?v=qVoH2QwwDWg&t=...,"[83, 103]"
521,niets anders kon,102,103,omdat ie op dat moment niets anders kon bedenken,dw4uOU5fUEY,https://www.youtube.com/watch?v=dw4uOU5fUEY&t=...,"[109, 167, 186]"
522,komt alleen omdat,431,433,is gekomen komt alleen omdat verschillende ond...,xKNDkrAQ3iM,https://www.youtube.com/watch?v=xKNDkrAQ3iM&t=...,"[110, 93, 138]"
523,zeg gewoon bedankt,549,551,zeg gewoon bedankt,UltPsmBVTV0,https://www.youtube.com/watch?v=UltPsmBVTV0&t=...,"[130, 121, 170]"


In [74]:
df_youtube_sent_sample.drop("word_index", axis=1, inplace=True)

In [75]:
df_youtube_sent_sample.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Sample.xlsx", index=False)

##### Selected Result Word Count

In [76]:
#df_selected_file = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment2/Result/6-Youtube Sentence Pos Tag Analysis/Deploy Result Manuel/{lang_folder.capitalize()}_200_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample Revised.xlsx")
df_selected_file = pd.read_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Sample.xlsx")
df_selected_file

,search_string,start_time,end_time,sentence,video_id,video_url
0,de n,291,294,tegenwoordig de n,EHLs51vEPuU,https://www.youtube.com/watch?v=EHLs51vEPuU&t=...
1,ik s,20,22,dan zet ik s nachts om,EL02PyHRmOU,https://www.youtube.com/watch?v=EL02PyHRmOU&t=20s
2,u de,94,95,wanneer u de comfort seat niet gebruikt moet h...,7KsIU2XtYo8,https://www.youtube.com/watch?v=7KsIU2XtYo8&t=94s
3,of t,19,20,het rijksmonument sur meuse bijvoorbeeld of t ...,mMvNkD7AJEk,https://www.youtube.com/watch?v=mMvNkD7AJEk&t=19s
4,ja u,129,130,ja u ziet het goed generaal maatennaaier is terug,3fAvv1RgbUQ,https://www.youtube.com/watch?v=3fAvv1RgbUQ&t=...
...,...,...,...,...,...,...
520,moeten misschien,178,179,dus klanten moeten misschien eventjes hun fiet...,qVoH2QwwDWg,https://www.youtube.com/watch?v=qVoH2QwwDWg&t=...
521,niets anders kon,102,103,omdat ie op dat moment niets anders kon bedenken,dw4uOU5fUEY,https://www.youtube.com/watch?v=dw4uOU5fUEY&t=...
522,komt alleen omdat,431,433,is gekomen komt alleen omdat verschillende ond...,xKNDkrAQ3iM,https://www.youtube.com/watch?v=xKNDkrAQ3iM&t=...
523,zeg gewoon bedankt,549,551,zeg gewoon bedankt,UltPsmBVTV0,https://www.youtube.com/watch?v=UltPsmBVTV0&t=...


In [77]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [78]:
df_shared_word_manuel_count = word_count_result(df_selected_file, df_selected_file.columns[0:1])
df_shared_word_manuel_count

,word,word_count
0,aan,7
1,oh,7
2,nee,7
3,net,7
4,niet,7
...,...,...
180,tegen,1
181,praten,1
182,gezien,1
183,niets,1


In [79]:
df_shared_word_manuel_count.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Count.xlsx", index=False)

##### Not In Sent And Selected Word List

In [80]:
#lang_folder = "Turkish"
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,ik,22539531
1,je,20769946
2,het,15696161
3,de,15258816
4,dat,13387137
...,...,...
331769,oorlogsgezichten,5
331770,opdrachtenlijst,5
331771,verlsaafde,5
331772,oxidatieproces,5


In [81]:
#word_start = 0
#word_end = 200
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select

,word,frequency
0,ik,22539531
1,je,20769946
2,het,15696161
3,de,15258816
4,dat,13387137
...,...,...
195,genoeg,353128
196,wist,352948
197,oh,351988
198,klaar,350506


In [82]:
def index_to_word(index_list, df_word, word_column, reverse=True):
    '''
    index_to_word(not_in_sent, df_word_select, "word", reverse=True)\n
    not in sent is index list as numeric\n
    df_word_select is a dataframe that includes word colunm ("word" is a column)\n
    reverse is used for string index list convert numeric.
    '''
    if reverse:
        word_index_dict = {var1:var2 for (var1, var2) in enumerate(df_word[f"{word_column}"])}
    else:
        word_index_dict = {var2:var1 for (var1, var2) in enumerate(df_word[f"{word_column}"])}

    convert_word_list = []
    for index in index_list:
        convert_word = word_index_dict[index]
        convert_word_list.append(convert_word)
    return convert_word_list        

In [83]:
not_in_sent_word = index_to_word(not_in_sent, df_word_select, "word", reverse=True)
not_in_sent_word

['geef',
 'helpen',
 'vind',
 'vrouw',
 'wist',
 'neem',
 'mr',
 'z',
 'spijt',
 'moeder',
 'vader',
 'doet',
 'wacht',
 'dacht',
 'bent']

In [84]:
set_not_in_sent_word = set(not_in_sent_word)

In [85]:
set_word = set(df_word_select["word"])
set_select_word = set(df_shared_word_manuel_count["word"])

In [86]:
set_not_in_select = set_word.difference(set_select_word)
set_not_in_select

{'bent',
 'dacht',
 'doet',
 'geef',
 'helpen',
 'moeder',
 'mr',
 'neem',
 'spijt',
 'vader',
 'vind',
 'vrouw',
 'wacht',
 'wist',
 'z'}

In [87]:
not_in_sent_selected_word_list = list(set_not_in_select.union(set_not_in_sent_word))
not_in_sent_selected_word_list

['mr',
 'vader',
 'z',
 'geef',
 'doet',
 'bent',
 'neem',
 'wist',
 'vind',
 'helpen',
 'moeder',
 'wacht',
 'spijt',
 'dacht',
 'vrouw']

In [88]:
len(not_in_sent_selected_word_list)

15

In [89]:
df_youtube_sent_select.drop(["word_index_list"], axis=1, inplace=True)
df_youtube_sent_select

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,search_string,start_time,end_time,sentence,video_id,video_url
0,de n,291,294,tegenwoordig de n,EHLs51vEPuU,https://www.youtube.com/watch?v=EHLs51vEPuU&t=...
1,ik s,20,22,dan zet ik s nachts om,EL02PyHRmOU,https://www.youtube.com/watch?v=EL02PyHRmOU&t=20s
2,u de,94,95,wanneer u de comfort seat niet gebruikt moet h...,7KsIU2XtYo8,https://www.youtube.com/watch?v=7KsIU2XtYo8&t=94s
3,of t,19,20,het rijksmonument sur meuse bijvoorbeeld of t ...,mMvNkD7AJEk,https://www.youtube.com/watch?v=mMvNkD7AJEk&t=19s
4,ja u,129,130,ja u ziet het goed generaal maatennaaier is terug,3fAvv1RgbUQ,https://www.youtube.com/watch?v=3fAvv1RgbUQ&t=...
...,...,...,...,...,...,...
127523,we hebben echt heel veel mensen nodig om dit t...,392,397,we hebben echt heel veel mensen nodig om dit t...,g01V_H0TU8o,https://www.youtube.com/watch?v=g01V_H0TU8o&t=...
127524,dus ja ik denk zeker dat het heel anders is en...,265,269,dus ja ik denk zeker dat het heel anders is en...,eldUSfVfYMc,https://www.youtube.com/watch?v=eldUSfVfYMc&t=...
127525,die we nu gaan doen maar ik wilde er wel even ...,234,240,die we nu gaan doen maar ik wilde er wel even ...,CqaNlQx_CmE,https://www.youtube.com/watch?v=CqaNlQx_CmE&t=...
127526,dat er voor ja moet er geen gaan we nou ja maa...,694,700,jarenlang dat er voor ja moet er geen gaan we ...,9rsOGla4V6s,https://www.youtube.com/watch?v=9rsOGla4V6s&t=...


In [90]:
def word_in_wordgroup(source_word_list, df_target, target_column, word_sample_num, simple=False):

    '''word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5, simple=False)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number.
       simple use for all column result or only target column result 
    '''
    if simple:
        df_select = df_target[[f"{target_column}"]].dropna()
    else:
        df_select = df_target
        
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,"word",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [91]:
if youtube_sent_usage == "all": 
    df_youtube_usage_select = df_youtube_sent_result
else:
    df_youtube_usage_select = df_youtube_sent_select

In [92]:
not_in_sent_selected_word_list_result = word_in_wordgroup(not_in_sent_selected_word_list, df_youtube_usage_select, "search_string", 5, simple=False)
not_in_sent_selected_word_list_result

,word,search_string,start_time,end_time,sentence,video_id,video_url
0,mr,wat de mr,214,216,die mij interesseert en vooral wat de mr betre...,eJjFkHKBX2g,https://www.youtube.com/watch?v=eJjFkHKBX2g&t=...
1,mr,van de mr werd op,165,170,voorzitter van de mr werd op,k9BO5Rvu6m4,https://www.youtube.com/watch?v=k9BO5Rvu6m4&t=...
2,vader,vader is,107,109,samenwonen vader is weduwnaar de zoon is een j...,4EGmtJWcZRA,https://www.youtube.com/watch?v=4EGmtJWcZRA&t=...
3,vader,en vader,538,539,en vader verliet daarna het huis,27wk3NFkSXs,https://www.youtube.com/watch?v=27wk3NFkSXs&t=...
4,vader,vader van,76,78,bert mansholt vader van sicco mansholt,k_RC6yJ5jKk,https://www.youtube.com/watch?v=k_RC6yJ5jKk&t=76s
...,...,...,...,...,...,...,...
67,vrouw,de vrouw,24,25,de vrouw bleef echter aandringen en ten slotte...,dQiqy78Pd2w,https://www.youtube.com/watch?v=dQiqy78Pd2w&t=24s
68,vrouw,vrouw die,11,13,schrijfster een fantastische vrouw die fascine...,sKvfuVWiqkI,https://www.youtube.com/watch?v=sKvfuVWiqkI&t=11s
69,vrouw,vrouw van,15,20,leuke vrouw van eind,8NhrEHB55e0,https://www.youtube.com/watch?v=8NhrEHB55e0&t=15s
70,vrouw,een vrouw,170,171,geval wanneer een vrouw consequent en achteree...,rnd3Id2vnp8,https://www.youtube.com/watch?v=rnd3Id2vnp8&t=...


In [93]:
#not_in_sent_selected_word_list_merge = pd.merge(not_in_sent_selected_word_list_result, df_youtube_sent_select, how="left", on="search_string")
#not_in_sent_selected_word_list_merge.drop_duplicates(inplace=True)
#not_in_sent_selected_word_list_merge

In [94]:
not_in_sent_selected_word_list_result.word.nunique()

15

In [95]:
set_not_in_sent_selected_word_list = set(not_in_sent_selected_word_list)
set_not_in_sent_selected_word_list_result = set(not_in_sent_selected_word_list_result["word"].to_list())

In [96]:
set_not_in_sent_selected_word_list.difference(set_not_in_sent_selected_word_list_result)

set()

In [97]:
#word_in_wordgroup_simple(["pekala","gece"], df_youtube_sent_select, "search_string", 5)

In [98]:
not_in_sent_selected_word_list_result["video_url"] = "https://www.youtube.com/watch?v="+not_in_sent_selected_word_list_result['video_id'].map(str)+"&t="+not_in_sent_selected_word_list_result['start_time'].map(str)+"s"
not_in_sent_selected_word_list_result

,word,search_string,start_time,end_time,sentence,video_id,video_url
0,mr,wat de mr,214,216,die mij interesseert en vooral wat de mr betre...,eJjFkHKBX2g,https://www.youtube.com/watch?v=eJjFkHKBX2g&t=...
1,mr,van de mr werd op,165,170,voorzitter van de mr werd op,k9BO5Rvu6m4,https://www.youtube.com/watch?v=k9BO5Rvu6m4&t=...
2,vader,vader is,107,109,samenwonen vader is weduwnaar de zoon is een j...,4EGmtJWcZRA,https://www.youtube.com/watch?v=4EGmtJWcZRA&t=...
3,vader,en vader,538,539,en vader verliet daarna het huis,27wk3NFkSXs,https://www.youtube.com/watch?v=27wk3NFkSXs&t=...
4,vader,vader van,76,78,bert mansholt vader van sicco mansholt,k_RC6yJ5jKk,https://www.youtube.com/watch?v=k_RC6yJ5jKk&t=76s
...,...,...,...,...,...,...,...
67,vrouw,de vrouw,24,25,de vrouw bleef echter aandringen en ten slotte...,dQiqy78Pd2w,https://www.youtube.com/watch?v=dQiqy78Pd2w&t=24s
68,vrouw,vrouw die,11,13,schrijfster een fantastische vrouw die fascine...,sKvfuVWiqkI,https://www.youtube.com/watch?v=sKvfuVWiqkI&t=11s
69,vrouw,vrouw van,15,20,leuke vrouw van eind,8NhrEHB55e0,https://www.youtube.com/watch?v=8NhrEHB55e0&t=15s
70,vrouw,een vrouw,170,171,geval wanneer een vrouw consequent en achteree...,rnd3Id2vnp8,https://www.youtube.com/watch?v=rnd3Id2vnp8&t=...


In [99]:
not_in_sent_selected_word_list_result.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Sample2.xlsx", index=False)

#### Copy Move And Delete

In [100]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_*.xlsx")
output_file

['Dutch_200_Word_Group_In_Youtube_Sentence_Sample.xlsx',
 'Dutch_200_Word_Group_In_Youtube_Sentence_Word_Count.xlsx',
 'Dutch_200_Word_Group_In_Youtube_Sentence_Sample2.xlsx']

In [101]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [102]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass